---
# Import from the API (ergast)


In [ ]:
import requests

def get_drivers_standings(year='current'):
    url = f'http://ergast.com/api/f1/{year}/driverStandings.json'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        standings = data['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']
        
        for position, driver_info in enumerate(standings, start=1):
            driver = driver_info['Driver']
            name = f"{driver['givenName']} {driver['familyName']}"
            points = driver_info['points']
            print(f"{position}. {name} - {points} points")
    else:
        print("Failed to retrieve data")

get_drivers_standings()

# Import Data from API to train the model

In [20]:
# http://ergast.com/api/f1/current/last/results

import requests
import json

url = 'http://ergast.com/api/f1/current/last/results.json'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    # Save data as json file
    with open('data.json', 'w') as file:
        json.dump(data, file)

    results = data['MRData']['RaceTable']['Races']

    print(results)

[{'season': '2023', 'round': '22', 'url': 'https://en.wikipedia.org/wiki/2023_Abu_Dhabi_Grand_Prix', 'raceName': 'Abu Dhabi Grand Prix', 'Circuit': {'circuitId': 'yas_marina', 'url': 'http://en.wikipedia.org/wiki/Yas_Marina_Circuit', 'circuitName': 'Yas Marina Circuit', 'Location': {'lat': '24.4672', 'long': '54.6031', 'locality': 'Abu Dhabi', 'country': 'UAE'}}, 'date': '2023-11-26', 'time': '13:00:00Z', 'Results': [{'number': '1', 'position': '1', 'positionText': '1', 'points': '26', 'Driver': {'driverId': 'max_verstappen', 'permanentNumber': '33', 'code': 'VER', 'url': 'http://en.wikipedia.org/wiki/Max_Verstappen', 'givenName': 'Max', 'familyName': 'Verstappen', 'dateOfBirth': '1997-09-30', 'nationality': 'Dutch'}, 'Constructor': {'constructorId': 'red_bull', 'url': 'http://en.wikipedia.org/wiki/Red_Bull_Racing', 'name': 'Red Bull', 'nationality': 'Austrian'}, 'grid': '1', 'laps': '58', 'status': 'Finished', 'Time': {'millis': '5222624', 'time': '1:27:02.624'}, 'FastestLap': {'rank'

---
# Getting the missing rounds

In [49]:
import requests
import json

start_round = 13
url = f'http://ergast.com/api/f1/2023/{start_round}/results.json'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    # Save data as json file
    with open('data.json', 'w') as file:
        json.dump(data, file)

In [50]:
import pandas as pd

# Define the round numbers
start_round = 13
end_round = 21

# Initialize an empty list to store the results
all_results = []

# import the existing results.csv file
last_results = pd.read_csv('/Users/louisar/Documents/DevVsCode/Efrei/F1-ML-Prediction/data/Formula 1 World Championship (1950 - 2023) Kaggle/results.csv')
races = pd.read_csv('../data/api_ergast/2024-02-06_12h16/races.csv')
last_drivers = pd.read_csv('../data/api_ergast/2024-02-06_12h16/drivers.csv')
last_constructors = pd.read_csv('../data/api_ergast/2024-02-06_12h16/constructors.csv')

final_results = last_results.copy()

# Loop over the range of round numbers
for round_number in range(start_round, end_round + 1):
    # Define the URL
    url = f'http://ergast.com/api/f1/2023/{round_number}/results.json'
    
    # Make the request
    response = requests.get(url)
    
    # Check the status code
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Extract the results
        results = data['MRData']['RaceTable']['Races']
        raceName = data['MRData']['RaceTable']['Races'][0]['raceName']

        
        # Lets start by getting the last resultId from the existing csv
        last_resultId = last_results.iloc[-1]['resultId']
        last_resultId += 1

        # We then need to get the raceId by matching raceName to the list of races from the last_races csv and get the most recent one
        raceId = races[races['name'] == raceName].iloc[-1]['raceId']

        data = []

        for race in results:
            for result in race['Results']:
                code = result['Driver'].get('code', None)
                permanentNumber = result['Driver'].get('permanentNumber', None)
                constructorId = result['Constructor'].get('constructorId', None)
                driverNumber = result.get('number', None)
                gridPosition = result.get('grid', None)
                positionOrder = result.get('position', None)
                points = result.get('points', None)
                laps = result.get('laps', None)
                time = result['Time'].get("time", None) if 'Time' in result else None
                milliseconds = result['Time'].get("millis", None) if 'Time' in result else None
                fastestLap = result['FastestLap'].get("lap", None) if 'FastestLap' in result else None
                rankFastestLap = result['FastestLap'].get("rank", None) if 'FastestLap' in result else None
                fastestLapTime = result['FastestLap']['Time'].get("time", None) if 'FastestLap' in result and 'Time' in result['FastestLap'] else None
                fastestLapSpeed = result['FastestLap']['AverageSpeed'].get("speed", None) if 'FastestLap' in result and 'AverageSpeed' in result['FastestLap'] else None
                status = result.get('status', None)

                # Get driverId from last_drivers and match code with code AND permanentNumber with number
                driver_df = last_drivers[(last_drivers['code'] == code) & (last_drivers['number'] == permanentNumber)]
                if not driver_df.empty:
                    driverId = driver_df.iloc[0]['driverId']
                else:
                    print(f"No driver found with code {code} and number {permanentNumber}, setting to 0")
                    driverId = 0

                # Get constructor id from last_constructors and match constructorId with constructorRef
                constructor_df = last_constructors[last_constructors['constructorRef'] == constructorId]
                if not constructor_df.empty:
                    constructorId = constructor_df.iloc[0]['constructorId']
                else:
                    print(f"No constructor found with constructorId {constructorId}, setting to 0")
                    constructorId = 0

                data.append({
                    'resultId': last_resultId,
                    'raceId': raceId,
                    'driverId': driverId,
                    'constructorId': constructorId,
                    'number': driverNumber,
                    'grid': gridPosition,
                    'position': positionOrder,
                    'positionText': positionOrder,
                    'positionOrder': positionOrder,
                    'points': points,
                    'laps': laps,
                    'time': time,
                    'milliseconds': milliseconds,
                    'fastestLap': fastestLap,
                    'rank': rankFastestLap,
                    'fastestLapTime': fastestLapTime,
                    'fastestLapSpeed': fastestLapSpeed,
                    'statusId': status
                })

                last_resultId += 1

        # Convert the list of dictionaries into a DataFrame
        df = pd.DataFrame(data)

        # Now add all data from df to last_results
        final_results = pd.concat([final_results, df], ignore_index=True)
    else:
        print(f"Failed to get results for round {round_number}")

# save final_results as csv
final_results.to_csv('results.csv', index=False)

No driver found with code LAW and number 40, setting to 0
No driver found with code LAW and number 40, setting to 0
No driver found with code LAW and number 40, setting to 0
No driver found with code LAW and number 40, setting to 0
No driver found with code LAW and number 40, setting to 0


---

In [36]:
import pandas as pd
import os
import glob
import requests
import json
from datetime import datetime

# We read the most recent results.csv and races.csv files
path_pattern = os.path.join("../data/api_ergast", "*")
directories = glob.glob(path_pattern)
most_recent_dir = max(directories, key=os.path.getmtime)
print("Checking latest data from: ",most_recent_dir)

# Importing the files
last_results = pd.read_csv(os.path.join(most_recent_dir + "/results.csv"))
last_races = pd.read_csv(os.path.join(most_recent_dir + "/races.csv"))

# We import the latest results from the api
url = 'http://ergast.com/api/f1/current/last/results.json'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    results = data['MRData']['RaceTable']['Races']
    raceName = data['MRData']['RaceTable']['Races'][0]['raceName']

# Loop through the API results
for race in results:
    raceId = last_races[last_races['name'] == raceName].iloc[-1]['raceId']

    # We check if this raceId already exists in the last_results DataFrame
    if raceId in last_results['raceId'].values:
        print("No new data.")
        break
    else:
        print("New data available.")
        # We then save the new data !

        # Lets start by getting the last resultId from the existing csv
        last_resultId = last_results.iloc[-1]['resultId']
        last_resultId += 1

        # We then need to get the raceId by matching raceName to the list of races from the last_races csv and get the most recent one
        raceId = last_races[last_races['name'] == raceName].iloc[-1]['raceId']

        data = []

        for race in results:
            for result in race['Results']:
                driverId = result['Driver'].get('driverId', None)
                constructorId = result['Constructor'].get('constructorId', None)
                driverNumber = result.get('number', None)
                gridPosition = result.get('grid', None)
                positionOrder = result.get('position', None)
                points = result.get('points', None)
                laps = result.get('laps', None)
                time = result['Time'].get("time", None) if 'Time' in result else None
                milliseconds = result['Time'].get("millis", None) if 'Time' in result else None
                fastestLap = result['FastestLap'].get("lap", None) if 'FastestLap' in result else None
                rankFastestLap = result['FastestLap'].get("rank", None) if 'FastestLap' in result else None
                fastestLapTime = result['FastestLap']['Time'].get("time", None) if 'FastestLap' in result and 'Time' in result['FastestLap'] else None
                fastestLapSpeed = result['FastestLap']['AverageSpeed'].get("speed", None) if 'FastestLap' in result and 'AverageSpeed' in result['FastestLap'] else None
                status = result.get('status', None)

                data.append({
                    'resultId': last_resultId,
                    'raceId': raceId,
                    'driverId': driverId,
                    'constructorId': constructorId,
                    'number': driverNumber,
                    'grid': gridPosition,
                    'position': positionOrder,
                    'positionText': positionOrder,
                    'positionOrder': positionOrder,
                    'points': points,
                    'laps': laps,
                    'time': time,
                    'milliseconds': milliseconds,
                    'fastestLap': fastestLap,
                    'rank': rankFastestLap,
                    'fastestLapTime': fastestLapTime,
                    'fastestLapSpeed': fastestLapSpeed,
                    'statusId': status
                })

                last_resultId += 1

        # Convert the list of dictionaries into a DataFrame
        df = pd.DataFrame(data)

        # Now add all data from df to last_results
        last_results = pd.concat([last_results, df], ignore_index=True)

        # Save to directory
        
        # Get current date and time
        current_date = datetime.now().strftime('%Y-%m-%d_%Hh%M')

        # Create new directory
        new_dir = f'../data/api_ergast/{current_date}'
        os.makedirs(new_dir, exist_ok=True)

        # Save the file in the new directory
        last_results.to_csv(f'{new_dir}/results.csv', index=False)

        print(f"\n--> Results saved in {new_dir} directory")

        # Also add the races.csv file
        last_races.to_csv(f'{new_dir}/races.csv', index=False)
    

Checking latest data from:  ../data/api_ergast/2024-02-07_00h13
No new data.


In [26]:
# Initialize an empty list to store the data
data = []

# Loop through the races
for race in results:
    driverId = race['Results'][0]['Driver']['driverId']
    constructorId = race['Results'][0]['Constructor']['constructorId']
    driverNumber = race['Results'][0]['number']
    gridPosition = race['Results'][0]['grid']
    positionOrder = race['Results'][0]['position']
    points = race['Results'][0]['points']
    laps = race['Results'][0]['laps']
    time = race['Results'][0]['Time']["time"]
    milliseconds = race['Results'][0]['Time']["millis"]
    fastestLap = race['Results'][0]['FastestLap']["lap"]
    rankFastestLap = race['Results'][0]['FastestLap']["rank"]
    fastestLapTime = race['Results'][0]['FastestLap']["Time"]["time"]
    fastestLapSpeed = race['Results'][0]['FastestLap']["AverageSpeed"]["speed"]
    status = race['Results'][0]['status']

    data.append({
        'resultId': 1,
        'raceId': 1,
        'driverId': driverId,
        'constructorId': constructorId,
        'number': driverNumber,
        'grid': gridPosition,
        'position': positionOrder,
        'positionText': positionOrder,
        'positionOrder': positionOrder,
        'points': points,
        'laps': laps,
        'time': time,
        'milliseconds': milliseconds,
        'fastestLap': fastestLap,
        'rank': rankFastestLap,
        'fastestLapTime': fastestLapTime,
        'fastestLapSpeed': fastestLapSpeed,
        'statusId': status
    })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)
df

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,1,max_verstappen,red_bull,1,1,1,1,1,26,58,1:27:02.624,5222624,45,1,1:26.993,218.541,Finished


---
# Import dataset

## *This dataset is updated from season 1950 to 2023*

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)
pd.get_option('display.max_columns', None)

In [6]:
path = "../data/Formula 1 World Championship (1950 - 2023) Kaggle"

# Load data
df_circuits = pd.read_csv(path + "/circuits.csv")
df_constructor_results = pd.read_csv(path + "/constructor_results.csv")
df_constructor_standings = pd.read_csv(path + "/constructor_standings.csv")
df_constructors = pd.read_csv(path + "/constructors.csv")
df_driver_standings = pd.read_csv(path + "/driver_standings.csv")
df_drivers = pd.read_csv(path + "/drivers.csv")
df_lap_times = pd.read_csv(path + "/lap_times.csv")
df_pit_stops = pd.read_csv(path + "/pit_stops.csv")
df_qualifying = pd.read_csv(path + "/qualifying.csv")
df_races = pd.read_csv(path + "/races.csv")
df_results = pd.read_csv(path + "/results.csv")
df_seasons = pd.read_csv(path + "/seasons.csv")
df_sprint_results = pd.read_csv(path + "/sprint_results.csv")
df_status = pd.read_csv(path + "/status.csv")

In [8]:
# find in df results all the results for driverid 832 and for positionOrder 1
df_results[(df_results['driverId'] == 832) & (df_results['positionOrder'] == 1)]

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
25580,25586,1083,832,6,55,1,1,1,1,25.0,52,2:17:50.311,8270311,44,2,1:30.813,233.530,1


In [4]:
df_status.head(8)

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine
5,6,Gearbox
6,7,Transmission
7,8,Clutch


### Limit data to last X seasons

In [5]:
min_year = 2019

In [6]:
# Limit df_races to rows where year is equal or greater to 2022
print("Len before filter:", len(df_races))
df_races = df_races[df_races['year'] >= min_year]
print("Len after filter:", len(df_races))

Len before filter: 1101
Len after filter: 104


In [7]:
# Now we can filter all other data to only keep the races from 2022 and 2023
print("> df_constructor_results")
print("Len before filter:", len(df_constructor_results))
df_constructor_results = df_constructor_results[df_constructor_results['raceId'].isin(df_races['raceId'])]
print("Len after filter:", len(df_constructor_results))

print("\n> df_constructor_standings")
print("Len before filter:", len(df_constructor_standings))
df_constructor_standings = df_constructor_standings[df_constructor_standings['raceId'].isin(df_races['raceId'])]
print("Len after filter:", len(df_constructor_standings))

print("\n> df_driver_standings")
print("Len before filter:", len(df_driver_standings))
df_driver_standings = df_driver_standings[df_driver_standings['raceId'].isin(df_races['raceId'])]
print("Len after filter:", len(df_driver_standings))

print("\n> df_lap_times")
print("Len before filter:", len(df_lap_times))
df_lap_times = df_lap_times[df_lap_times['raceId'].isin(df_races['raceId'])]
print("Len after filter:", len(df_lap_times))

print("\n> df_pit_stops")
print("Len before filter:", len(df_pit_stops))
df_pit_stops = df_pit_stops[df_pit_stops['raceId'].isin(df_races['raceId'])]
print("Len after filter:", len(df_pit_stops))

print("\n> df_qualifying")
print("Len before filter:", len(df_qualifying))
df_qualifying = df_qualifying[df_qualifying['raceId'].isin(df_races['raceId'])]
print("Len after filter:", len(df_qualifying))

print("\n> df_results")
print("Len before filter:", len(df_results))
df_results = df_results[df_results['raceId'].isin(df_races['raceId'])]
print("Len after filter:", len(df_results))  

print("\n> df_sprint_results")
print("Len before filter:", len(df_sprint_results))
df_sprint_results = df_sprint_results[df_sprint_results['raceId'].isin(df_races['raceId'])]
print("Len after filter:", len(df_sprint_results))

> df_constructor_results
Len before filter: 12290
Len after filter: 940

> df_constructor_standings
Len before filter: 13051
Len after filter: 940

> df_driver_standings
Len before filter: 34124
Len after filter: 1937

> df_lap_times
Len before filter: 551742
Len after filter: 102863

> df_pit_stops
Len before filter: 10089
Len after filter: 3302

> df_qualifying
Len before filter: 9815
Len after filter: 1877

> df_results
Len before filter: 26080
Len after filter: 1880

> df_sprint_results
Len before filter: 180
Len after filter: 180


---
# Data Exploration

> ## Circuits

In [8]:
print("List of countries: \n",
      df_circuits['country'].unique()
)

print("\nList of locations: \n",
      df_circuits['location'].unique()
)

List of countries: 
 ['Australia' 'Malaysia' 'Bahrain' 'Spain' 'Turkey' 'Monaco' 'Canada'
 'France' 'UK' 'Germany' 'Hungary' 'Belgium' 'Italy' 'Singapore' 'Japan'
 'China' 'Brazil' 'USA' 'United States' 'UAE' 'Argentina' 'Portugal'
 'South Africa' 'Mexico' 'Korea' 'Netherlands' 'Sweden' 'Austria'
 'Morocco' 'Switzerland' 'India' 'Russia' 'Azerbaijan' 'Saudi Arabia'
 'Qatar']

List of locations: 
 ['Melbourne' 'Kuala Lumpur' 'Sakhir' 'Montmeló' 'Istanbul' 'Monte-Carlo'
 'Montreal' 'Magny Cours' 'Silverstone' 'Hockenheim' 'Budapest' 'Valencia'
 'Spa' 'Monza' 'Marina Bay' 'Oyama' 'Shanghai' 'São Paulo' 'Indianapolis'
 'Nürburg' 'Imola' 'Suzuka' 'Las Vegas' 'Abu Dhabi' 'Buenos Aires'
 'Jerez de la Frontera' 'Estoril' 'Okayama' 'Adelaide' 'Midrand'
 'Castle Donington' 'Mexico City' 'Phoenix' 'Le Castellet'
 'Yeongam County' 'Rio de Janeiro' 'Detroit' 'Kent' 'Zandvoort'
 'Heusden-Zolder' 'Dijon' 'Dallas' 'California' 'Nevada' 'Madrid'
 'New York State' 'Anderstorp' 'Ontario' 'Barcelona' 'Bru

> ## Constructors

In [9]:
df_constructors.head(5)

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


> ## Drivers

In [10]:
df_drivers.head(5)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


> ## Races

In [11]:
df_races.tail(5)

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
1096,1116,2023,18,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00
1097,1117,2023,19,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N
1098,1118,2023,20,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00
1099,1119,2023,21,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-17,04:30:00,2023-11-17,08:00:00,2023-11-18,04:30:00,2023-11-18,08:00:00,\N,\N
1100,1120,2023,22,24,Abu Dhabi Grand Prix,2023-11-26,13:00:00,https://en.wikipedia.org/wiki/2023_Abu_Dhabi_G...,2023-11-24,09:30:00,2023-11-24,13:00:00,2023-11-25,10:30:00,2023-11-25,14:00:00,\N,\N


In [12]:
print("Number of races by year (order from most to less): \n",)
print(df_races.groupby('year').count()['raceId'].sort_values(ascending=False))

Number of races by year (order from most to less): 

year
2021    22
2022    22
2023    22
2019    21
2020    17
Name: raceId, dtype: int64


# Visualizations

### Average positions by driver

In [14]:
# Average finishing positions for drivers
driver_avg_positions = df_results.groupby('driverId')['positionOrder'].mean().reset_index()
driver_avg_positions.columns = ['driverId', 'avgPosition']

# Standard deviation of finishing positions for drivers
# In simple terms, it shows how much the drivers' finishing positions vary
driver_position_std = df_results.groupby('driverId')['positionOrder'].std().reset_index()
driver_position_std.columns = ['driverId', 'positionStd']

# Merge the average and standard deviation
driver_performance = pd.merge(driver_avg_positions, driver_position_std, on='driverId')

# Merge with drivers data to add the driver's name
driver_performance = pd.merge(driver_performance, df_drivers[['driverId', 'driverRef']], on='driverId')

# Print first rows ordered by lowest average position
driver_performance = driver_performance.sort_values(by='avgPosition', ascending=True)
driver_performance.head()

,driverId,avgPosition,positionStd,driverRef
0,1,3.655914,3.812151,hamilton
12,830,4.340426,5.746534,max_verstappen
18,844,7.170213,5.819209,leclerc
7,815,7.326087,5.507455,perez
9,822,8.106383,6.144266,bottas


### Everage Pit Stops by driver

In [15]:
# Average number of pit stops per race for each driver
avg_pit_stops = df_pit_stops.groupby(['raceId', 'driverId']).size().reset_index(name='pitStops')
avg_pit_stops = avg_pit_stops.groupby('driverId')['pitStops'].mean().reset_index()

# Add driver's name
avg_pit_stops = pd.merge(avg_pit_stops, df_drivers[['driverId', 'driverRef']], on='driverId')

# Print first rows ordered by lowest average pit stops
avg_pit_stops = avg_pit_stops.sort_values(by='pitStops', ascending=True)

avg_pit_stops.head()

,driverId,pitStops,driverRef
5,154,1.558824,grosjean
8,817,1.703704,ricciardo
14,839,1.718750,ocon
6,807,1.756757,hulkenberg
10,825,1.772727,kevin_magnussen


### Lap Times Analysis

In [16]:
df_lap_times.head(5)

,raceId,driverId,lap,position,time,milliseconds
448879,1010,822,1,1,1:33.813,93813
448880,1010,822,2,1,1:29.662,89662
448881,1010,822,3,1,1:29.246,89246
448882,1010,822,4,1,1:28.641,88641
448883,1010,822,5,1,1:28.759,88759


In [20]:
# Define a function to convert 'mm:ss.ms' to seconds
def convert_time(time_str):
    minutes, seconds = time_str.split(':')
    return float(minutes) * 60 + float(seconds)

# Apply this function to the 'time' column
df_lap_times['time'] = df_lap_times['time'].apply(convert_time)

# Now you can calculate the mean and variance
avg_lap_times = df_lap_times.groupby(['raceId', 'driverId'])['time'].mean().reset_index()
lap_time_variance = df_lap_times.groupby(['raceId', 'driverId'])['time'].var().reset_index()
lap_time_variance.columns = ['raceId', 'driverId', 'lapTimeVariance']

# Merge and analyze
lap_time_analysis = pd.merge(avg_lap_times, lap_time_variance, on=['raceId', 'driverId'])

# Add driver's name
lap_time_analysis = pd.merge(lap_time_analysis, df_drivers[['driverId', 'driverRef']], on='driverId')

# Print first rows ordered by lowest average lap time
lap_time_analysis = lap_time_analysis.sort_values(by='time', ascending=True)

print(lap_time_analysis.head())

     raceId  driverId       time  lapTimeVariance driverRef
713    1046       815  62.932345       124.395230     perez
719    1046       839  63.053241       114.210862      ocon
720    1046       840  63.068770       113.507454    stroll
726    1046       849  63.070115       103.283559    latifi
718    1046       832  63.076943       110.522124     sainz


### Status Analysis

In [22]:
# Most common reasons for not finishing a race
not_finished_reasons = df_status[df_status['statusId'] != 1]
common_reasons = not_finished_reasons['status'].value_counts().reset_index()
common_reasons.columns = ['reason', 'count']

print(common_reasons.head(10))

               reason  count
0        Disqualified      1
1            Excluded      1
2       Not restarted      1
3          Alternator      1
4         Underweight      1
5         Safety belt      1
6            Oil pump      1
7           Fuel leak      1
8  Did not prequalify      1
9         Spark plugs      1
